In [ ]:
from sif_src.loader import download_ms_marco, load_data, save_processed_data
from sif_src.preprocess_data import preprocess_text

#download Marco

data = load_data('data/raw/ms_marco.csv')

data['processed_text'] = data['text'].apply(preprocess_text)

save_processed_data(data, 'data/processed/ms_marco_processed.csv')


In [ ]:
from sif_src.utils import load_word_vectors
from sif_src.sif import compute_word_frequencies, compute_sif_weights, compute_sif_embeddings, remove_pc


word_vectors = load_word_vectors('path_to_word_vectors.txt')

corpus = data['processed_text'].tolist()

word_freq = compute_word_frequencies(corpus)
sif_weights = compute_sif_weights(word_freq)

embeddings = compute_sif_embeddings(corpus, word_vectors, sif_weights)

final_embeddings = remove_pc(embeddings)
